<a href="https://colab.research.google.com/github/TDMSundriyal/Projects/blob/main/Face_Recognizer/Source_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mysql.connector

In [ ]:
mydb=mysql.connector.connect(
host="localhost",
user="root",
passwd=""
) 

In [ ]:
#print(mydb)
#mycursor=mydb.cursor()
#mycursor.execute("Drop database customer")

In [ ]:
mycursor=mydb.cursor()
#mycursor.execute("create database students")
mycursor.execute("show databases")
for x in mycursor:
    print(x)

('authorized_user',)
('customer',)
('information_schema',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('python_database',)
('students',)


In [ ]:
#mycursor=mydb.cursor()
#mycursor.execute("create database Customer")

In [ ]:
mydb=mysql.connector.connect(
host="localhost",
user="root",
passwd="",
database="customer"
) 

In [ ]:
#mycursor=mydb.cursor()
#mycursor.execute("create table cust_data(id int primary key,Name varchar(20),Address varchar(50),Age int,Country varchar(15),Sex varchar(5),Mode varchar(8),Comment varchar(5000))")

In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import scrolledtext
import cv2
import os
from PIL import Image
import numpy as np

In [ ]:
window=tk.Tk()
window.title("Face recognition system")
window.config(background="chartreuse")
l1=tk.Label(window,text="Name",font=("trebuchet ms",20),bg="chartreuse")
l1.grid(column=0, row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1, row=0)

l2=tk.Label(window,text="Age",font=("trebuchet ms",20),bg="chartreuse")
l2.grid(column=0, row=1)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1, row=1)

l3=tk.Label(window,text="Address",font=("trebuchet ms",20),bg="chartreuse")
l3.grid(column=0, row=2)
t3=tk.Entry(window,width=50,bd=5)
t3.grid(column=1, row=2)

#COMBOBOX
l4=tk.Label(window,text="COUNTRY",font=("trebuchet ms",20),bg="chartreuse")
l4.place(x=0,y=150)

from tkinter.ttk import *
c1=Combobox(window)
c1['values']=("India","Nepal","US","UK","USSR","UAE","others",)
c1.current(0)
c1.place(x=150,y=150)

#CHECKBUTTON
l5=tk.Label(window,text="MODE(s)",font=("trebuchet ms",20),bg="chartreuse")
l5.place(x=0,y=200)
var1=tk.IntVar()
c2=Checkbutton(window,text="Online",var=var1)
var2=tk.IntVar()
c3=Checkbutton(window,text="Offline",var=var2)

c2.place(x=200,y=200)
c3.place(x=260,y=200)

#RADIOBUTTON
l6=tk.Label(window,text="SEX",font=("trebuchet ms",20),bg="chartreuse")
l6.place(x=0,y=250)
var3=tk.IntVar()
r1=tk.Radiobutton(window,text='Male',value=1,var=var3)
r2=tk.Radiobutton(window,text='Female',value=2,var=var3)
r3=tk.Radiobutton(window,text='Others',value=3,var=var3)
r1.place(x=200,y=250)
r2.place(x=260,y=250)
r3.place(x=330,y=250)

#scrolledText
l7=tk.Label(window,text="COMMENT",font=("trebuchet ms",20),bg="chartreuse")
l7.place(x=0,y=300)

txt=scrolledtext.ScrolledText(window,width=30,height=1)
txt.place(x=150,y=305)
txt.insert(tk.INSERT,'write some comment')


#b=tk.Button(window,text="SUBMIT",font=("Arial Bold",20),bg="orange",fg="red",command=btn_clicked)
#b.place(x=150,y=380)

def train_classifier():
    data_dir="D:/Face_Recognizer/Data"
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces  = []
    ids   = []
    
    for image in path:
        img = Image.open(image).convert('L');
        imageNp= np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)
    
    #Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result','Training dataset completed!!!')
    
b1=tk.Button(window,text="Training",font=("trebuchet ms",20),bg='aqua',fg='gray',command=train_classifier)
b1.place(x=0,y=450)

def detect_face():
    def draw_boundary(img,classifier,scaleFactor,minNeighbors,color,text,clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbors)

        coords = []

        for(x,y,w,h) in features:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            id,pred = clf.predict(gray_image[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))
            
            mydb=mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            database="customer"
            )
            mycursor=mydb.cursor()
            mycursor.execute("select name from cust_data where id="+str(id))
            s = mycursor.fetchone()
            s = ''+''.join(s)
            
            if confidence>74:
                cv2.putText(img,s,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)   
            else:
                cv2.putText(img,"UNKNOWN",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),1,cv2.LINE_AA)

            coords=[x,y,w,h]
        return coords
            
    def recognize(img,clf,faceCascade):
        coords = draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
        return img

    faceCascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()  #Local Binary Pattern Histogram
    clf.read("classifier.xml")

    video_capture =  cv2.VideoCapture(0)

    while True:
        ret,img = video_capture.read()
        img=  recognize(img,clf,faceCascade)
        cv2.imshow("face detection",img)

        if cv2.waitKey(1)==13:
            break

    video_capture.release()
    cv2.destroyAllWindows()

b2=tk.Button(window,text="Detect the face",font=("trebuchet ms",20),bg='green1',fg='gray',command=detect_face)
b2.place(x=0,y=520)

def generate_dataset():
    if(t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Result','Please provide complete details of the user')
    else:
        mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="customer"
        )
        mycursor=mydb.cursor()
        mycursor.execute("SELECT * from cust_data")
        myresult=mycursor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql="insert into cust_data(id,name,age,address,country,mode,sex,comment) values(%s,%s,%s,%s,%s,%s,%s,%s)"
        
        #to get checkButton values
        if var1.get()==1 and var2.get()==1:
            s1="Blend Mode"
        elif var1.get()==1:
            s1="ONLINE"
        elif var2.get()==1:
            s1="OFFLINE"
        else:
            s1=""

        #to get radioButton values
        if var1.get()==1:
            s2="MALE"
        elif var1.get()==2:
            s2="FEMALE"
        elif var1.get()==3:
            s2="OTHERS"
        else:
            s2=""

        val=(id,t1.get(),t2.get(),t3.get(),c1.get(),s1,s2,txt.get("1.0",tk.END))
        mycursor.execute(sql,val)
        mydb.commit()
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray,1.3,5)
            #scaling factor=1.3
            #Minimum neighbor = 5

            if faces is ():
                return None
            for(x,y,w,h) in faces:
                cropped_face=img[y:y+h,x:x+w]
            return cropped_face

        cap = cv2.VideoCapture(0)
        img_id=0

        while True:
            ret,frame = cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face = cv2.resize(face_cropped(frame),(200,200))
                face  = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path,face)
                cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1, (0,255,0),2)
                # (50,50) is the origin point from where text is to be written
                # font scale=1
                #thickness=2

                cv2.imshow("Cropped face",face)
                if cv2.waitKey(1)==13 or int(img_id)==200:
                    break
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result','Generating dataset completed!!!')

b3=tk.Button(window,text="Generate dataset",font=("trebuchet ms",20),bg='yellow1',fg='gray',command=generate_dataset)
b3.place(x=0,y=380)

window.geometry("500x600")
window.mainloop()

<>:191: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:191: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\tripa\AppData\Local\Temp\ipykernel_17640\1354685231.py:191: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
